In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# 1. Đọc và cắt dữ liệu lịch sử chỉ số ngành (từ 10/4/2020 đến 9/4/2025)
stocks = {
    'MBB': 'MBB.csv',
    'BID': 'Dữ liệu Lịch sử BID 245.4T.csv',
    'VCB': 'Dữ liệu Lịch sử VCB 481.29T.csv',
    'CTG': 'Dữ liệu Lịch sử CTG 201.91T.csv',
    'TCB': 'Dữ liệu Lịch sử TCB 190.04T.csv'
}
capitalization = {'MBB': 141.88, 'BID': 245.4, 'VCB': 481.29, 'CTG': 201.91, 'TCB': 190.04}  # Đơn vị: nghìn tỷ

data_dict = {}
for stock, file in stocks.items():
    df = pd.read_csv(file)
    df['Ngày'] = pd.to_datetime(df['Ngày'], format='%d/%m/%Y')
    df = df[(df['Ngày'] >= pd.to_datetime('2020-04-10')) & (df['Ngày'] < pd.to_datetime('2025-04-10'))]
    df['Lần cuối'] = df['Lần cuối'].astype(str).str.replace(',', '').astype(float)
    df = df.rename(columns={'Ngày': 'Date', 'Lần cuối': f'{stock}_Price'})
    data_dict[stock] = df[['Date', f'{stock}_Price']]

# Ghép dữ liệu
industry_data = data_dict['MBB'].copy()
for stock in ['BID', 'VCB', 'CTG', 'TCB']:
    industry_data = industry_data.merge(data_dict[stock], on='Date', how='left')
industry_data = industry_data.ffill()

# Tính chỉ số ngành
total_cap = sum(capitalization.values())
industry_data['Banking_Industry_Index'] = 0
for stock in stocks.keys():
    industry_data['Banking_Industry_Index'] += industry_data[f'{stock}_Price'] * (capitalization[stock] / total_cap)
industry_data = industry_data[['Date', 'Banking_Industry_Index']]

# Lưu dữ liệu lịch sử
industry_data.to_csv('Banking_Industry_Historical.csv', index=False)
print("Dữ liệu lịch sử chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Historical.csv'")
print(f"Dữ liệu lịch sử có {len(industry_data)} ngày, từ {industry_data['Date'].min()} đến {industry_data['Date'].max()}")

# 2. Dự đoán chỉ số ngành bằng Prophet (theo tháng)
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.fit(industry_data.rename(columns={'Date': 'ds', 'Banking_Industry_Index': 'y'}))

# Tạo khung thời gian tương lai theo tháng (từ 1/4/2025 đến 1/5/2028)
future_dates_monthly = pd.date_range(start='2025-04-01', end='2028-05-01', freq='MS')
future_df_monthly = pd.DataFrame({'ds': future_dates_monthly})
forecast = model_prophet.predict(future_df_monthly)
industry_monthly = forecast[['ds', 'yhat']]
industry_monthly.columns = ['Date', 'Banking_Industry_Index']
industry_monthly['Date'] = pd.to_datetime(industry_monthly['Date'])
print(f"Dự đoán Prophet có {len(industry_monthly)} tháng, từ {industry_monthly['Date'].min()} đến {industry_monthly['Date'].max()}")
industry_monthly.to_csv('Banking_Industry_Monthly_Forecast.csv', index=False)
print("Dự đoán đầu tháng chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Monthly_Forecast.csv'")

# 3. Chuẩn bị dữ liệu ngày cho LSTM
all_dates = pd.date_range(start='2020-04-10', end='2025-04-09', freq='D')
data_day = pd.DataFrame({'Date': all_dates})
data_day = data_day.merge(industry_data, on='Date', how='left')
data_day['Banking_Industry_Index'] = data_day['Banking_Industry_Index'].ffill()
data_day = data_day[['Date', 'Banking_Industry_Index']]

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
scaled_data_day = scaler.fit_transform(data_day['Banking_Industry_Index'].values.reshape(-1, 1))
if np.any(np.isnan(scaled_data_day)) or np.any(np.isinf(scaled_data_day)):
    scaled_data_day = np.nan_to_num(scaled_data_day, nan=0.0, posinf=0.0, neginf=0.0)

# 4. Chuẩn bị dữ liệu cho LSTM
time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data_day, time_step)
X_train_tensor = torch.FloatTensor(X).unsqueeze(-1)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 5. Định nghĩa và huấn luyện mô hình LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 30
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 6. Dự đoán từ 10/4/2025 đến 1/5/2028 với scale liên tục
historical_data = scaled_data_day.flatten().tolist()
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})
daily_df = daily_df.merge(industry_monthly, on='Date', how='left')

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]  # Lấy cửa sổ cuối cùng từ dữ liệu thực tế
last_scaled_value = scaler.inverse_transform([[historical_data[-1]]])[0, 0]

for i in range(len(industry_monthly) - 1):
    start_date = max(industry_monthly['Date'].iloc[i], pd.to_datetime('2025-04-10'))
    end_date = industry_monthly['Date'].iloc[i + 1] - pd.Timedelta(days=1)
    next_month_start = industry_monthly['Date'].iloc[i + 1]
    if i == len(industry_monthly) - 2:  # Đoạn cuối cùng kéo dài đến 1/5/2028
        end_date = pd.to_datetime('2028-05-01')

    if start_date > end_date:
        continue

    segment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    n_days = len(segment_dates)
    if n_days == 0:
        continue

    segment_predictions = []
    current_window_tensor = torch.FloatTensor([current_window[-time_step:]]).unsqueeze(-1).to(device)
    with torch.no_grad():
        window = current_window_tensor
        for _ in range(n_days):
            pred = model_lstm(window).cpu().numpy()[0, 0]
            segment_predictions.append(pred)
            new_window = np.append(window.cpu().numpy()[0, 1:, 0], pred)
            window = torch.FloatTensor(new_window).unsqueeze(0).unsqueeze(-1).to(device)

    segment_predictions = np.array(segment_predictions).reshape(-1, 1)
    segment_predictions_rescaled = scaler.inverse_transform(segment_predictions)

    if end_date >= next_month_start - pd.Timedelta(days=1):
        monthly_price_next = industry_monthly['Banking_Industry_Index'].iloc[i + 1]
        daily_price_at_end = segment_predictions_rescaled[-1, 0]
        daily_price_at_start = segment_predictions_rescaled[0, 0]

        adjustment_at_start = last_scaled_value - daily_price_at_start
        adjustment_at_end = monthly_price_next - daily_price_at_end

        segment_length = n_days
        adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
        segment_predictions_scaled = segment_predictions_rescaled + adjustments.reshape(-1, 1)
        last_scaled_value = segment_predictions_scaled[-1, 0]
    else:
        adjustment_at_start = last_scaled_value - daily_price_at_start
        segment_predictions_scaled = segment_predictions_rescaled + adjustment_at_start
        last_scaled_value = segment_predictions_scaled[-1, 0]

    scaled_segment = scaler.transform(segment_predictions_scaled.reshape(-1, 1)).flatten()
    current_window.extend(scaled_segment.tolist())
    predictions_daily.extend(segment_predictions_scaled.flatten())

# Tạo DataFrame kết hợp
combined_data = pd.concat([industry_data[industry_data['Date'] <= pd.to_datetime('2025-04-09')], 
                          pd.DataFrame({'Date': future_dates_daily, 'Banking_Industry_Index': predictions_daily})])
combined_data = combined_data[(combined_data['Date'] >= pd.to_datetime('2020-04-10')) & 
                              (combined_data['Date'] <= pd.to_datetime('2028-05-01'))]
combined_data = combined_data.reset_index(drop=True)

# Lưu kết quả
combined_data.to_csv('Banking_Industry_Daily_Predictions.csv', index=False)
print("Dự đoán ngày chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Daily_Predictions.csv'")
print(f"Dữ liệu dự đoán ngày cuối cùng: {combined_data['Date'].max()}")

# 7. Biểu đồ minh họa
plt.figure(figsize=(14, 7))
plt.plot(industry_data['Date'], industry_data['Banking_Industry_Index'], label='Lịch sử', color='blue')
plt.plot(combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Date'], 
         combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Banking_Industry_Index'], 
         label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.plot(industry_monthly['Date'], industry_monthly['Banking_Industry_Index'], label='Dự đoán (Tháng)', color='orange', marker='o')
plt.title('Dự đoán Chỉ Số Ngành Ngân Hàng (2020-2028)')
plt.xlabel('Ngày')
plt.ylabel('Chỉ Số Ngành')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('Banking_Industry_Prediction_Chart.png')
plt.close()
print("Biểu đồ dự đoán chỉ số ngành đã được lưu vào 'Banking_Industry_Prediction_Chart.png'")

18:37:28 - cmdstanpy - INFO - Chain [1] start processing


Dữ liệu lịch sử chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Historical.csv'
Dữ liệu lịch sử có 1249 ngày, từ 2020-04-10 00:00:00 đến 2025-04-09 00:00:00


18:37:29 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Tuan Anh\AppData\Local\Temp\ipykernel_17856\3738493995.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry_monthly['Date'] = pd.to_datetime(industry_monthly['Date'])


Dự đoán Prophet có 38 tháng, từ 2025-04-01 00:00:00 đến 2028-05-01 00:00:00
Dự đoán đầu tháng chỉ số ngành ngân hàng đã được lưu vào 'Banking_Industry_Monthly_Forecast.csv'
Epoch [1/30], Loss: 0.018020
Epoch [2/30], Loss: 0.007766
Epoch [3/30], Loss: 0.003225
Epoch [4/30], Loss: 0.002200
Epoch [5/30], Loss: 0.002402
Epoch [6/30], Loss: 0.007689
Epoch [7/30], Loss: 0.005449
Epoch [8/30], Loss: 0.001867
Epoch [9/30], Loss: 0.001097
Epoch [10/30], Loss: 0.003360
Epoch [11/30], Loss: 0.001307
Epoch [12/30], Loss: 0.001774
Epoch [13/30], Loss: 0.001534
Epoch [14/30], Loss: 0.001164
Epoch [15/30], Loss: 0.001069
Epoch [16/30], Loss: 0.001624
Epoch [17/30], Loss: 0.000848
Epoch [18/30], Loss: 0.001554
Epoch [19/30], Loss: 0.001782
Epoch [20/30], Loss: 0.000624
Epoch [21/30], Loss: 0.001019
Epoch [22/30], Loss: 0.001864
Epoch [23/30], Loss: 0.001363
Epoch [24/30], Loss: 0.000721
Epoch [25/30], Loss: 0.000915
Epoch [26/30], Loss: 0.000889
Epoch [27/30], Loss: 0.001246
Epoch [28/30], Loss: 0.000